In [49]:
import tensorflow as tf 
from tensorflow.keras.applications import ResNet50 
from tensorflow.keras.layers import GlobalAveragePooling2D , Dense , Lambda ,Input
from tensorflow.keras.models import  Model
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.applications.resnet50 import preprocess_input  
import numpy as np 
import os 

In [50]:
base_model = ResNet50(weights='imagenet',include_top=False, input_shape=(224,224,3))


In [51]:
def create_feature():
    input_layer = Input(shape=(224,224,3))
    x = base_model(input_layer)
    x= GlobalAveragePooling2D()(x)
    return Model(input_layer,x)

feature_extractor = create_feature()   

In [52]:
def build_siamese_network():
    input_1 = Input(shape=(224,224,3))
    input_2 = Input(shape=(224,224,3))
    
    feature_1 = feature_extractor(input_1)
    feature_2 = feature_extractor(input_2)

    l2_distance = Lambda(lambda tensors : tf.keras.backend.abs(tensors[0]-tensors[1]))([feature_1,feature_2])
    x = Dense(512,activation='relu')(l2_distance)
    x = Dense(128,activation='relu')(x)

    output = Dense(3,activation='softmax')(x)

    siamese_model = Model(inputs=[input_1,input_2],outputs=output)

    siamese_model.compile(optimizer='adam',loss= 'categorical_crossentropy' ,metrics=['accuracy'])
    
    return siamese_model


In [53]:
siamese_model =  build_siamese_network()
siamese_model.summary()

Model: "functional_33"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_35      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_36      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_31       │ (None, 2048)      │ 23,587,712 │ input_layer_35[0… │
│ (Functional)        │                   │            │ input_layer_36[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_2 (Lambda)   │ (None, 2048)      │          0 │ functional_31[0]… │
│                     │                   │            │ functional_31[1]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_42 (Dense)    │ (None, 512)       │  1,049,088 │ lambda_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_43 (Dense)    │ (None, 128)       │     65,664 │ dense_42[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_44 (Dense)    │ (None, 3)         │        387 │ dense_43[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 24,702,851 (94.23 MB)

 Trainable params: 24,649,731 (94.03 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [54]:
img_path_1 = 'Volunteers Data\MADAM REHANA\hand (after 15 days).jpg'  # First image path (before)
img_path_2 = 'Volunteers Data\MADAM REHANA\hand (after 35 days).jpg'  # Second image path (after)

img1 = image.load_img(img_path_1, target_size=(224, 224))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1, axis=0)
img1 = preprocess_input(img1)

img2 = image.load_img(img_path_2, target_size=(224, 224))
img2 = image.img_to_array(img2)
img2 = np.expand_dims(img2, axis=0)
img2 = preprocess_input(img2)

# Predict whether the skin quality has improved, deteriorated, or remained the same
prediction = siamese_model.predict([img1, img2])

# Interpret the prediction
class_labels = ['improved', 'no change', 'deteriorated']
predicted_class = class_labels[np.argmax(prediction[0])]
print(f"Prediction: Skin quality has {predicted_class}.")


<>:1: SyntaxWarning: invalid escape sequence '\M'
<>:2: SyntaxWarning: invalid escape sequence '\M'
<>:1: SyntaxWarning: invalid escape sequence '\M'
<>:2: SyntaxWarning: invalid escape sequence '\M'
C:\Users\arham\AppData\Local\Temp\ipykernel_4056\2853168960.py:1: SyntaxWarning: invalid escape sequence '\M'
  img_path_1 = 'Volunteers Data\MADAM REHANA\hand (after 15 days).jpg'  # First image path (before)
C:\Users\arham\AppData\Local\Temp\ipykernel_4056\2853168960.py:2: SyntaxWarning: invalid escape sequence '\M'
  img_path_2 = 'Volunteers Data\MADAM REHANA\hand (after 35 days).jpg'  # Second image path (after)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Prediction: Skin quality has deteriorated.


In [55]:
#adding pipeline 
class ImagePairPipeline:
    def __init__(self,model,image_size =(224,224,3),preprocess_func =preprocess_input):
        self.model = model 
        self.image_size = image_size
        self.preprocess_func = preprocess_func
        self.class_labels = ['improved', 'no change', 'deteriorated']

    def load_and_preprocess_image(self,img_path):
        img = image.load_img(img_path,target_size =self.image_size)
        img = image.img_to_array(img)
        img = np.expand_dims(img ,axis=0)
        img = self.preprocess_func(img)
        return img 
    
    def predict(self , img_path_1 , img_path_2):
        img1 = self.load_and_preprocess_image(img_path_1)
        img2 = self.load_and_preprocess_image(img_path_2)

        prediction = self.model.predict([img1,img2])
        return prediction 
    
    def predict_on_pairs(self,img_paths):
        predictions = []
        for i in range(len(img_paths)-1):
            pred = self.predict(img_paths[i],img_paths[i+1])
            predictions.append((img_paths[i], img_paths[i + 1], pred))
        return predictions  






        

In [56]:
img_dir = 'Volunteers Data/MADAM REHANA'
img_paths = [
    os.path.join(img_dir, 'hand (after 15 days).jpg'),
    os.path.join(img_dir, 'hand (after 35 days).jpg'),
    
]

pipeline = ImagePairPipeline(siamese_model)

predictions = pipeline.predict_on_pairs(img_paths)

for img1, img2, pred in predictions:
    class_labels = ['improved', 'no change', 'deteriorated']
    predicted_class = class_labels[np.argmax(pred)]
    print(f"Prediction between {os.path.basename(img1)} and {os.path.basename(img2)}: Skin quality has {predicted_class}.")





1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
Prediction between hand (after 15 days).jpg and hand (after 35 days).jpg: Skin quality has deteriorated.


In [57]:
img_dir = 'Volunteers Data/MADAM REHANA'
img_paths = [
    os.path.join(img_dir, 'head (after 15 days).jpg'),
    os.path.join(img_dir, 'head (after 35 days).jpg'),
    os.path.join(img_dir, 'head 2 (after 35 days).jpg'),

    
]

pipeline = ImagePairPipeline(siamese_model)

predictions = pipeline.predict_on_pairs(img_paths)

for img1, img2, pred in predictions:
    class_labels = ['improved', 'no change', 'deteriorated']
    predicted_class = class_labels[np.argmax(pred)]
    print(f"Prediction between {os.path.basename(img1)} and {os.path.basename(img2)}: Skin quality has {predicted_class}.")





1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Prediction between head (after 15 days).jpg and head (after 35 days).jpg: Skin quality has deteriorated.
Prediction between head (after 35 days).jpg and head 2 (after 35 days).jpg: Skin quality has deteriorated.


In [58]:
img_dir = 'Volunteers Data/MADAM REHANA'
img_paths = [
    os.path.join(img_dir, 'L HEAD(0 time).jpg'),
    os.path.join(img_dir, 'L HEAD 2(0 time).jpg'),
]

pipeline = ImagePairPipeline(siamese_model)

predictions = pipeline.predict_on_pairs(img_paths)

for img1, img2, pred in predictions:
    class_labels = ['improved', 'no change', 'deteriorated']
    predicted_class = class_labels[np.argmax(pred)]
    print(f"Prediction between {os.path.basename(img1)} and {os.path.basename(img2)}: Skin quality has {predicted_class}.")





1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
Prediction between L HEAD(0 time).jpg and L HEAD 2(0 time).jpg: Skin quality has deteriorated.
